In [3]:
!curl -o /tmp/hadoop-aws-3.3.4.jar https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar
!curl -o /tmp/aws-java-sdk-bundle-1.12.262.jar https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  940k  100  940k    0     0  2605k      0 --:--:-- --:--:-- --:--:-- 2611k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  267M  100  267M    0     0  10.7M      0  0:00:24  0:00:24 --:--:-- 10.1M


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("spark-test") \
    .config("spark.master", "local") \
    .config('spark.driver.cores', '1')\
    .config('spark.driver.memory', '1G')\
    .config('spark.executor.instances', '1')\
    .config('spark.executor.cores', '1')\
    .config('spark.executor.memory', '512m')\
    .config("spark.ui.port", "4045")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars", "/tmp/hadoop-aws-3.3.4.jar,/tmp/aws-java-sdk-bundle-1.12.262.jar") \
    .config("spark.hadoop.fs.defaultFS", "s3a://ozone-om") \
    .config("spark.hadoop.fs.s3a.access.key", "ozone-access-key") \
    .config("spark.hadoop.fs.s3a.secret.key", "ozone-secret-key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://ozone-s3g:9878") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.committer.name", "directory") \
    .config("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging") \
    .config("spark.hadoop.fs.s3a.multiobjectdelete.enable", "false") \
    .config("spark.hadoop.fs.s3a.directory.marker.retention", "keep") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "200000") \
    .config("spark.hadoop.fs.s3a.socket.timeout", "200000") \
    .config("spark.hadoop.fs.s3a.attempts.maximum", "50") \
    .config("spark.hadoop.fs.s3a.retry.limit", "50") \
    .config("spark.hadoop.fs.s3a.retry.interval", "5s") \
    .getOrCreate()
    #.enableHiveSupport() \
    #.config("spark.hadoop.fs.s3a.committer.name", "magic") \
    #.config("spark.hadoop.fs.s3a.committer.magic.enabled", "true") \
    #.config("spark.sql.warehouse.dir", "ofs://ozone-om/s3v/mybucket/warehouse/") \
    #.config("spark.sql.catalogImplementation", "hive")\
    #.config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")\

26/02/01 16:14:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.sparkContext.uiWebUrl

'http://8f4f1f60e449:4045'

In [4]:
df = spark.createDataFrame([(1, "tgete", "2025-02-01"), (3, "vdsv", "2025-06-01")], ["id", "descr", "report_dt"])

In [5]:
df.show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [6]:
df.write\
    .mode("overwrite") \
    .format("parquet") \
    .save("s3a://mybucket/test_db/spark_test/test.parquet")

26/02/01 16:14:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
spark.read \
    .parquet("s3a://mybucket/test_db/spark_test/test.parquet") \
    .show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [8]:
spark.stop()